In [1]:
import pandas as pd
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.util import dataframe_utils


# process mining 
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery


# viz
from pm4py.visualization.petri_net import visualizer as pn_visualizer
from pm4py.visualization.process_tree import visualizer as pt_visualizer
from pm4py.visualization.heuristics_net import visualizer as hn_visualizer
from pm4py.visualization.dfg import visualizer as dfg_visualization


# misc 
from pm4py.objects.conversion.process_tree import converter as pt_converter


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [12]:
log_csv = pd.read_csv("data\BPI_Challenge_2012\BPI_Challenge_2012-training.csv", sep=',')
#log_csv = dataframe_utils.convert_timestamp_columns_in_df(log_csv)
#log_csv = log_csv.sort_values('event time:timestamp')
log_csv.rename(columns={'case concept:name': 'case:concept:name', 'case REG_DATE': 'case:REG_DATE', 
'case AMOUNT_REQ': 'case:AMOUNT_REQ', 'event concept:name' : 'event:concept:name', 'event lifecycle:transition': 'event:lifecycle:transition',
'event time:timestamp': 'event:time:timestamp'}, inplace=True)


In [4]:
event_log

NameError: name 'event_log' is not defined

In [7]:
import pm4py

In [13]:
event_log = pm4py.format_dataframe(log_csv, case_id='case:concept:name', activity_key='event:concept:name', timestamp_key='event:time:timestamp', timest_format="%Y-%m-%d %H:%M:%S%z")

In [ ]:
pm 

In [ ]:
event_log[0]

In [14]:
from pm4py.objects.log.util import interval_lifecycle
log = interval_lifecycle.to_interval(event_log)

AttributeError: 'DataFrame' object has no attribute 'attributes'

In [15]:
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
net, initial_marking, final_marking = alpha_miner.apply(event_log)

from pm4py.algo.conformance.tokenreplay import algorithm as token_replay
replayed_traces = token_replay.apply(event_log, net, initial_marking, final_marking)

replaying log with TBR, completed variants :: 100%|██████████| 4217/4217 [00:13<00:00, 307.81it/s]


In [ ]:
from pm4py.algo.filtering.log.start_activities import start_activities_filter
from pm4py.algo.filtering.log.end_activities import end_activities_filter


log_start = start_activities_filter.get_start_activities(event_log)
end_activities = end_activities_filter.get_end_activities(event_log)
log_start  # Printing the start activity in our log

In [ ]:
end_activities  # Printing the end activity in our log


In [ ]:
from pm4py.algo.filtering.log.variants import variants_filter
from pm4py.statistics.traces.generic.log import case_statistics
variants = variants_filter.get_variants(log)
  
print(f"We have {len(variants)} variants in our log")

In [ ]:
variants_count = case_statistics.get_variant_statistics(log)
variants_count = sorted(variants_count, key=lambda x: x['count'], reverse=True)
## Printing the top 10 variants by case number
variants_count[:10] 


In [ ]:
from pm4py.algo.filtering.log.attributes import attributes_filter
activities = attributes_filter.get_attribute_values(log, "concept:name")
activities


In [ ]:
import os 
os.environ["PATH"] += os.pathsep + "C:\Program Files (x86)\\Graphviz\\bin" # First install graphviz via the website, then run this code
# The program should be added to the PATH somewhere on your computer, there is a little instruction for this online.

In [16]:
net, initial_marking, final_marking = alpha_miner.apply(log)
gviz = pn_visualizer.apply(net, initial_marking, final_marking)
pn_visualizer.view(gviz)



## Adding frequency will make it more informative.

parameters = {pn_visualizer.Variants.FREQUENCY.value.Parameters.FORMAT: "png"}
gviz = pn_visualizer.apply(net, initial_marking, final_marking, 
                           parameters=parameters,
                           variant=pn_visualizer.Variants.FREQUENCY,
                           log=log)
pn_visualizer.view(gviz)

NameError: name 'log' is not defined

In [ ]:
tree = inductive_miner.apply_tree(log)

gviz = pt_visualizer.apply(tree)
pt_visualizer.view(gviz)

In [ ]:
net, initial_marking, final_marking = pt_converter.apply(tree, 
variant=pt_converter.Variants.TO_PETRI_NET)
gviz = pn_visualizer.apply(net, initial_marking, final_marking)
pn_visualizer.view(gviz)

In [ ]:
heu_net = heuristics_miner.apply_heu(log, 
parameters={heuristics_miner.Variants.CLASSIC.value.Parameters.DEPENDENCY_THRESH: 0.5})

gviz = hn_visualizer.apply(heu_net)
hn_visualizer.view(gviz)

In [ ]:
dfg = dfg_discovery.apply(log)


from pm4py.visualization.dfg import visualizer as dfg_visualization

gviz = dfg_visualization.apply(dfg, log=log, variant=dfg_visualization.Variants.FREQUENCY)
dfg_visualization.view(gviz)

In [ ]:
dfg = dfg_discovery.apply(log)


from pm4py.visualization.dfg import visualizer as dfg_visualization

gviz = dfg_visualization.apply(dfg, log=log, variant=dfg_visualization.Variants.PERFORMANCE)
dfg_visualization.view(gviz)